In [31]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars xgboost4j-spark-0.72.jar,xgboost4j-0.72.jar pyspark-shell'

import findspark
findspark.init()

import pyspark
from pyspark.sql.session import SparkSession
from pyspark.sql.types import *
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml import Pipeline
from pyspark.sql.functions import col

In [2]:
spark = SparkSession\
        .builder\
        .appName("PySpark XGBOOST Titanic")\
        .getOrCreate()

In [3]:
spark.sparkContext.addPyFile("YOUR_PATH/sparkxgb.zip")

In [4]:
from sparkxgb import XGBoostEstimator

In [5]:
schema = StructType(
  [StructField("PassengerId", DoubleType()),
    StructField("Survival", DoubleType()),
    StructField("Pclass", DoubleType()),
    StructField("Name", StringType()),
    StructField("Sex", StringType()),
    StructField("Age", DoubleType()),
    StructField("SibSp", DoubleType()),
    StructField("Parch", DoubleType()),
    StructField("Ticket", StringType()),
    StructField("Fare", DoubleType()),
    StructField("Cabin", StringType()),
    StructField("Embarked", StringType())
  ])

In [6]:
df_raw = spark\
  .read\
  .option("header", "true")\
  .schema(schema)\
  .csv("YOUR_PATH/train.csv")

In [13]:
df_raw.show(2)

+-----------+--------+------+--------------------+------+----+-----+-----+---------+-------+-----+--------+
|PassengerId|Survival|Pclass|                Name|   Sex| Age|SibSp|Parch|   Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+---------+-------+-----+--------+
|        1.0|     0.0|   3.0|Braund, Mr. Owen ...|  male|22.0|  1.0|  0.0|A/5 21171|   7.25| null|       S|
|        2.0|     1.0|   1.0|Cumings, Mrs. Joh...|female|38.0|  1.0|  0.0| PC 17599|71.2833|  C85|       C|
+-----------+--------+------+--------------------+------+----+-----+-----+---------+-------+-----+--------+
only showing top 2 rows



In [8]:
df = df_raw.na.fill(0)

In [9]:
sexIndexer = StringIndexer()\
  .setInputCol("Sex")\
  .setOutputCol("SexIndex")\
  .setHandleInvalid("keep")
    
cabinIndexer = StringIndexer()\
  .setInputCol("Cabin")\
  .setOutputCol("CabinIndex")\
  .setHandleInvalid("keep")
    
embarkedIndexer = StringIndexer()\
  .setInputCol("Embarked")\
  .setOutputCol("EmbarkedIndex")\
  .setHandleInvalid("keep")

In [10]:
vectorAssembler = VectorAssembler()\
  .setInputCols(["Pclass", "SexIndex", "Age", "SibSp", "Parch", "Fare", "CabinIndex", "EmbarkedIndex"])\
  .setOutputCol("features")

In [19]:
xgboost = XGBoostEstimator(
    featuresCol="features", 
    labelCol="Survival", 
    predictionCol="prediction"
)

In [20]:
pipeline = Pipeline().setStages([sexIndexer, cabinIndexer, embarkedIndexer, vectorAssembler, xgboost])

In [24]:
trainDF, testDF = df.randomSplit([0.8, 0.2], seed=24)

In [26]:
model = pipeline.fit(trainDF)

In [32]:
model.transform(testDF).select(col("PassengerId"), col("prediction")).show()

+-----------+----------+
|PassengerId|prediction|
+-----------+----------+
|        1.0|       0.0|
|        4.0|       1.0|
|       14.0|       0.0|
|       15.0|       1.0|
|       20.0|       1.0|
|       28.0|       1.0|
|       34.0|       0.0|
|       38.0|       0.0|
|       50.0|       1.0|
|       52.0|       0.0|
|       59.0|       1.0|
|       60.0|       0.0|
|       82.0|       0.0|
|       94.0|       0.0|
|       96.0|       0.0|
|       99.0|       1.0|
|      104.0|       0.0|
|      105.0|       0.0|
|      107.0|       1.0|
|      116.0|       0.0|
+-----------+----------+
only showing top 20 rows

